# Test technique – Diffusion pour la génération de microstructures matériaux

**Candidat :** [Votre nom]

**Date :** [Date]

---

## Instructions

Ce notebook contient la structure pour réaliser le test technique.

- Les sections marquées `# TODO` sont à compléter
- Les cellules de code vides sont à remplir
- N'hésitez pas à ajouter des cellules si nécessaire
- Commentez votre code

**Durée estimée :** 4 heures

---

## Imports et configuration

Imports de base déjà fournis. Vous pouvez en ajouter si nécessaire.

In [ ]:
# Imports standards
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
from skimage.draw import polygon
from skimage.segmentation import find_boundaries

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# TODO: Ajouter d'autres imports si nécessaire

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

---

# Partie 1 — Analyse exploratoire (30 min)

**Objectif :** Charger et comprendre le dataset de microstructures.

## 1.1 Chargement du dataset

In [ ]:
# TODO: Charger le fichier 'microstructures.npz'
data = # VOTRE CODE ICI

# TODO: Extraire M, G, O, Z du fichier
M = 
G = 
O = 
Z = 

# Afficher les dimensions
print(f"M shape: {M.shape}")
print(f"G shape: {G.shape}")
print(f"O shape: {O.shape}")
print(f"Z shape: {Z.shape}")
print(f"\nNombre d'échantillons: {M.shape[0]}")

## 1.2 Normalisation des données

**Important :** Les données doivent être normalisées pour l'entraînement.

In [ ]:
# TODO: Normaliser G (z-score normalization)
G = 

# TODO: Normaliser O (diviser par pi)
O = 

# TODO: Normaliser Z (z-score normalization par colonne)
Z = 

## 1.3 Visualisation de 5 microstructures aléatoires

In [ ]:
# TODO: Sélectionner 5 indices aléatoires
N = M.shape[0]
indices = # VOTRE CODE ICI

# TODO: Afficher les 5 microstructures (M)
# Conseil: utilisez plt.subplots ou plusieurs plt.figure

# VOTRE CODE ICI

## 1.4 Histogrammes de G et O

In [ ]:
# TODO: Tracer l'histogramme de G (aplatir en 1D d'abord)

# VOTRE CODE ICI

# TODO: Tracer l'histogramme de O

# VOTRE CODE ICI

## 1.5 Analyse des corrélations

In [ ]:
# TODO: Calculer la matrice de corrélation de Z
# Conseil: utilisez np.corrcoef

corr_matrix = # VOTRE CODE ICI

print("Matrice de corrélation de Z:")
print(corr_matrix)

# TODO (optionnel): Visualiser avec une heatmap

**Question :** Que remarquez-vous sur les corrélations ?

*Votre réponse ici*

---

# Partie 2 — Diffusion conditionnelle (1h30)

**Objectif :** Implémenter un modèle de diffusion conditionnel DDPM.

## 2.1 Paramètres de diffusion

Définir les paramètres du processus de diffusion (forward process).

In [ ]:
# TODO: Définir le nombre de timesteps T
T = # VOTRE VALEUR (conseil: 1000)

# TODO: Créer le schedule de beta (linéaire de 1e-4 à 0.02)
beta = # VOTRE CODE ICI

# TODO: Calculer alpha = 1 - beta
alpha = 

# TODO: Calculer alpha_bar = produit cumulé de alpha
alpha_bar = 

print(f"T = {T}")
print(f"beta range: [{beta.min():.6f}, {beta.max():.6f}]")
print(f"alpha_bar range: [{alpha_bar.min():.6f}, {alpha_bar.max():.6f}]")

## 2.2 Forward diffusion process

Implémente la fonction pour ajouter du bruit à une image.

In [ ]:
def q_sample(x0, t, noise):
    """
    Forward diffusion: ajoute du bruit à x0 au timestep t.
    
    Args:
        x0: image originale (batch, channels, H, W)
        t: timesteps (batch,)
        noise: bruit gaussien de même dimension que x0
    
    Returns:
        xt: image bruitée au timestep t
    """
    # TODO: Récupérer alpha_bar pour le timestep t
    # Conseil: alpha_bar[t] puis ajouter des dimensions pour le broadcasting
    a = # VOTRE CODE ICI
    
    # TODO: Calculer xt = sqrt(alpha_bar_t) * x0 + sqrt(1 - alpha_bar_t) * noise
    xt = # VOTRE CODE ICI
    
    return xt

## 2.3 Module FiLM (Feature-wise Linear Modulation)

Pour conditionner le modèle sur le vecteur z.

In [ ]:
class FiLM(nn.Module):
    """Module FiLM pour conditionner les features sur z."""
    
    def __init__(self, zdim, num_channels):
        """
        Args:
            zdim: dimension du vecteur de conditionnement z
            num_channels: nombre de canaux à conditionner
        """
        super().__init__()
        
        # TODO: Créer un MLP qui prend z et produit gamma et beta
        # Conseil: nn.Sequential avec Linear(zdim, 128), ReLU, Linear(128, 2*num_channels)
        self.net = # VOTRE CODE ICI
    
    def forward(self, x, z):
        """
        Args:
            x: features (batch, H, W, channels)
            z: vecteur de conditionnement (batch, zdim)
        
        Returns:
            features modulées
        """
        # TODO: Passer z dans le réseau et séparer en gamma et beta
        params = self.net(z)
        gamma, beta = # VOTRE CODE ICI (split en 2)
        
        # TODO: Appliquer FiLM: gamma * x + beta
        # Attention aux dimensions!
        output = # VOTRE CODE ICI
        
        return output

## 2.4 U-Net conditionnel

Architecture principale du modèle de diffusion.

In [ ]:
class CondUNet(nn.Module):
    """U-Net simple avec conditionnement FiLM."""
    
    def __init__(self, zdim=4):
        super().__init__()
        
        # TODO: Encoder - première couche
        # Conseil: Conv2d(3, 32, 3, padding=1)
        self.enc1 = # VOTRE CODE ICI
        
        # TODO: Encoder - deuxième couche  
        # Conseil: Conv2d(32, 64, 3, padding=1)
        self.enc2 = # VOTRE CODE ICI
        
        # TODO: Modules FiLM pour conditionner
        self.film1 = FiLM(zdim, 32)
        self.film2 = FiLM(zdim, 64)
        
        # TODO: Decoder - première couche
        # Conseil: ConvTranspose2d(64, 32, 3, padding=1)
        self.dec1 = # VOTRE CODE ICI
        
        # TODO: Couche de sortie
        # Conseil: Conv2d(32, 3, 3, padding=1) pour retourner 3 canaux
        self.out = # VOTRE CODE ICI
    
    def forward(self, x, z):
        """
        Args:
            x: image bruitée (batch, 3, 64, 64)
            z: vecteur de conditionnement (batch, zdim)
        
        Returns:
            bruit prédit (batch, 3, 64, 64)
        """
        # TODO: Encoder avec FiLM
        h1 = F.relu(self.enc1(x))
        h1 = # VOTRE CODE (appliquer film1)
        
        h2 = F.relu(self.enc2(h1))
        h2 = # VOTRE CODE (appliquer film2)
        
        # TODO: Decoder
        d1 = F.relu(self.dec1(h2))
        
        # TODO: Sortie
        output = # VOTRE CODE ICI
        
        return output

## 2.5 Préparation des données pour l'entraînement

In [ ]:
# TODO: Stacker M, G, O pour créer des images 3 canaux
# Conseil: np.stack([M, G, O], axis=1)
X = # VOTRE CODE ICI

# TODO: Convertir en tensors PyTorch
X_tensor = # VOTRE CODE ICI
Z_tensor = # VOTRE CODE ICI

# TODO: Déplacer sur le device
X_tensor = X_tensor.to(device)
Z_tensor = Z_tensor.to(device)

# TODO: Créer un TensorDataset et DataLoader
dataset = TensorDataset(X_tensor, Z_tensor)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Dataset size: {len(dataset)}")
print(f"Batch size: 32")
print(f"Number of batches: {len(loader)}")

## 2.6 Initialisation du modèle et optimiseur

In [ ]:
# TODO: Créer le modèle
model = CondUNet(zdim=4).to(device)

# TODO: Créer l'optimiseur (Adam avec lr=1e-3)
optimizer = # VOTRE CODE ICI

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2.7 Boucle d'entraînement

In [ ]:
# Configuration
num_epochs = 50
losses = []

print("Début de l'entraînement...\n")

for epoch in range(num_epochs):
    epoch_losses = []
    
    for x0, z in loader:
        # TODO: Générer un timestep aléatoire pour chaque échantillon du batch
        t = # VOTRE CODE ICI
        
        # TODO: Générer du bruit aléatoire de même dimension que x0
        noise = # VOTRE CODE ICI
        
        # TODO: Créer xt bruité avec q_sample
        xt = # VOTRE CODE ICI
        
        # TODO: Prédire le bruit avec le modèle
        pred_noise = # VOTRE CODE ICI
        
        # TODO: Calculer la loss MSE entre le bruit prédit et le vrai bruit
        loss = # VOTRE CODE ICI
        
        # TODO: Backpropagation
        optimizer.zero_grad()
        # VOTRE CODE ICI
        
        epoch_losses.append(loss.item())
    
    # Moyenne de la loss pour l'epoch
    avg_loss = np.mean(epoch_losses)
    losses.append(avg_loss)
    
    # Affichage tous les 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

print("\n✓ Entraînement terminé!")

## 2.8 Visualisation de la courbe d'apprentissage

In [ ]:
# TODO: Tracer la courbe de loss

# VOTRE CODE ICI

---

# Partie 3 — Évaluation physique (1h)

**Objectif :** Générer des microstructures et évaluer leur qualité.

## 3.1 Fonction de sampling (reverse diffusion)

In [ ]:
@torch.no_grad()
def sample(model, z, size=64):
    """
    Génère une microstructure à partir du modèle.
    
    Args:
        model: modèle entraîné
        z: vecteur de conditionnement (zdim,) ou (1, zdim)
        size: taille de l'image
    
    Returns:
        image générée (1, 3, size, size)
    """
    model.eval()
    
    # TODO: Commencer avec du pur bruit gaussien
    xt = # VOTRE CODE ICI
    
    # Assurer que z a la bonne dimension
    if z.dim() == 1:
        z = z.unsqueeze(0)
    
    # TODO: Boucle de débruitage (du timestep T-1 à 0)
    for t in reversed(range(T)):
        # TODO: Prédire le bruit avec le modèle
        predicted_noise = # VOTRE CODE ICI
        
        # Récupérer les coefficients
        alpha_t = alpha[t]
        alpha_bar_t = alpha_bar[t]
        
        # TODO: Étape de débruitage
        # Formule: xt = (1/sqrt(alpha_t)) * (xt - ((1-alpha_t)/sqrt(1-alpha_bar_t)) * predicted_noise)
        # Si t > 0, ajouter du bruit: + sqrt(beta_t) * noise
        
        if t > 0:
            noise = torch.randn_like(xt)
            beta_t = beta[t]
            xt = # VOTRE CODE ICI
        else:
            xt = # VOTRE CODE ICI (sans le terme de bruit)
    
    return xt

## 3.2 Fonctions de calcul des métriques

In [ ]:
def grain_stats(G_channel):
    """Calcule la moyenne et l'écart-type du canal G."""
    # TODO: Retourner (moyenne, std) du tenseur G_channel
    # Conseil: .mean().item() et .std().item()
    return # VOTRE CODE ICI

def orientation_stats(O_channel):
    """Calcule la moyenne du canal O."""
    # TODO: Retourner la moyenne du tenseur O_channel
    return # VOTRE CODE ICI

## 3.3 Génération et évaluation de 20 microstructures

In [ ]:
# Générer 20 échantillons et calculer les erreurs
num_samples = 20
errors = []

print("Génération de microstructures...\n")

for k in range(num_samples):
    # TODO: Générer un échantillon avec le vecteur z[k]
    xgen = # VOTRE CODE ICI
    
    # TODO: Extraire les canaux M, G, O (rappel: xgen a la forme (1, 3, 64, 64))
    # Canal 0: M, Canal 1: G, Canal 2: O
    M_gen = xgen[0, 0]
    G_gen = xgen[0, 1]
    O_gen = xgen[0, 2]
    
    # TODO: Calculer les statistiques
    muG, stdG = grain_stats(G_gen)
    muO = orientation_stats(O_gen)
    
    # TODO: Calculer les erreurs par rapport à Z[k]
    # Z contient [muG, sigmaG, muO, nb_voisins]
    err_muG = abs(muG - Z[k, 0])
    err_stdG = abs(stdG - Z[k, 1])
    err_muO = abs(muO - Z[k, 2])
    
    errors.append([err_muG, err_stdG, err_muO])

errors = np.array(errors)
print("✓ Génération terminée!")

## 3.4 Visualisation des erreurs

In [ ]:
# TODO: Tracer la courbe d'erreur moyenne pour chaque métrique
# Conseil: errors.mean(0) pour obtenir la moyenne sur les 20 échantillons

# VOTRE CODE ICI

# TODO: Afficher les valeurs numériques
print("\nErreurs moyennes:")
print(f"  μG (taille moyenne): {errors[:, 0].mean():.4f}")
print(f"  σG (écart-type): {errors[:, 1].mean():.4f}")
print(f"  μO (orientation): {errors[:, 2].mean():.4f}")

## 3.5 Visualisation d'échantillons générés

In [ ]:
# TODO: Générer et visualiser quelques exemples
# Afficher les 3 canaux (M, G, O) pour 3 échantillons

# VOTRE CODE ICI

**Analyse :** Commentez la qualité des microstructures générées.

*Votre analyse ici*

---

# Partie 4 — Question ouverte (1h)

**Question :** Proposez une stratégie pour générer des microstructures 3D cohérentes à partir de métriques 2D uniquement.

**Répondez dans la cellule markdown ci-dessous (sans coder).**

## Votre proposition

*Décrivez votre stratégie ici. Pensez à :*

*- Comment représenter une microstructure 3D ?*

*- Comment utiliser les métriques 2D pour contraindre la génération 3D ?*

*- Quelles architectures ou méthodes utiliser ?*

*- Quels sont les défis principaux ?*

---

*VOTRE RÉPONSE ICI (minimum 200 mots)*

---

---

# Bonus (optionnel)

Si vous avez du temps, améliorez votre solution :

- [ ] Ajouter des skip connections au U-Net
- [ ] Implémenter un timestep embedding
- [ ] Utiliser BatchNorm ou GroupNorm
- [ ] Tester différents learning rates
- [ ] Sauvegarder et charger le modèle
- [ ] Ajouter des visualisations supplémentaires

Documentez vos améliorations ci-dessous.

In [ ]:
# Code bonus (optionnel)

---

# Fin du test

**N'oubliez pas de fournir également un PDF (2 pages max) expliquant vos choix méthodologiques !**

**Points à aborder dans le PDF :**
- Architecture choisie et justification
- Hyperparamètres et pourquoi
- Difficultés rencontrées
- Pistes d'amélioration